In [193]:
import datetime
import hyperopt
import itertools
import math
import numpy
import operator
import pandas
import random
import requests
from IPython.display import Audio
from sklearn import (
    calibration,
    ensemble,
    linear_model,
    metrics,
    model_selection,
    naive_bayes,
    neighbors,
    neural_network,
    svm,
)
import time
import tqdm
import xgboost

### Read and Filter

In [207]:
dtype = {'smoke': float, 'alco': float, 'active': float}

def fix_ap(value):
    value = abs(value)
    while value > 500.0:
        value /= 10.0
    return value

def read_csv(filename):
    frame = pandas.read_csv(filename, sep=';', header=0, na_values='None', dtype=dtype).drop(['id'], axis=1)
    
    frame = pandas.get_dummies(frame, columns=[
        # 'smoke',
        # 'active',
        # 'alco',
        # 'gender',
        # 'cholesterol',
        # 'gluc',
    ])
    frame = frame.assign(
        bmi=(frame['weight'] / frame['height'] ** 2),
        # aged_smoke_0=(frame['smoke_0.0'] * frame['age']),
        # aged_smoke_1=(frame['smoke_1.0'] * frame['age']),
        # aged_active_0=(frame['active_0.0'] * frame['age']),
        # aged_active_1=(frame['active_1.0'] * frame['age']),
        # aged_alco_0=(frame['alco_0.0'] * frame['age']),
        # aged_alco_1=(frame['alco_1.0'] * frame['age']),
        # smoke_1_active_0=(frame['smoke_1.0'] * frame['active_0.0']),
    )
    frame = frame.drop([
        # 'smoke',
        # 'alco',
        # 'active',
    ], axis=1)
    
    frame['ap_hi'] = frame['ap_hi'].apply(fix_ap)
    
    return frame

In [208]:
train = read_csv('train.csv')

X = train.drop('cardio', axis=1).values
y = train['cardio'].values
print(f'X: {X.shape}')
print(f'y: {y.shape}')

X: (70000, 12)
y: (70000,)


### Helpers

In [286]:
def print_importances(estimator):
    for name, importance in sorted(zip(test, estimator.feature_importances_), key=operator.itemgetter(1), reverse=True):
        print(f'{name}: {importance:.7f}')

def make_int_grid(params, param_name, delta):
    n = params.get(param_name, 1)
    return {param_name: list(range(max(1, n - delta), n + delta + 1))}

def make_float_grid(params, param_name, delta, min_, max_):
    x = params.get(param_name, 0.0)
    return {param_name: [max(min_, x - delta), x, min(max_, x + delta)]}

def search(estimator, X, y, params, **param_grid):
    start_time = time.time()
    grid_search = model_selection.GridSearchCV(
        estimator.set_params(**params),
        param_grid,
        scoring='neg_log_loss',
        refit=False,
        cv=cv(),
    ).fit(X, y)
    print(f'[{(time.time() - start_time):.0f}s] {grid_search.best_score_:.7f} {grid_search.best_params_} | {param_grid}')
    return grid_search

def search_xgb(X, y, params):
    estimator = xgboost.XGBClassifier(nthread=5, seed=0)
    for _ in itertools.count(start=1):
        print()
        last_params = dict(params)
        params.update(search(estimator, X, y, params, **make_float_grid(params, 'base_score', 0.01, 0.0, 1.0)).best_params_)
        params.update(search(estimator, X, y, params, **make_float_grid(params, 'scale_pos_weight', 0.02, 0.0, 1.0)).best_params_)
        params.update(search(estimator, X, y, params, **make_float_grid(params, 'reg_lambda', 0.01, 0.0, 1.0)).best_params_)
        params.update(search(estimator, X, y, params, **make_float_grid(params, 'reg_alpha', 0.01, 0.0, 1.0)).best_params_)
        params.update(search(estimator, X, y, params, **make_float_grid(params, 'gamma', 0.01, 0.0, 1.0)).best_params_)
        params.update(search(estimator, X, y, params, **make_int_grid(params, 'max_depth', 1)).best_params_)
        params.update(search(estimator, X, y, params, **make_int_grid(params, 'n_estimators', 3)).best_params_)
        if params == last_params:
            break
    return estimator.set_params(**params).fit(X, y)

### CV

In [213]:
def cv():
    return model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

### Predict `smoke`, `alco` and `active`

In [296]:
X_helper = train.drop(['smoke', 'alco', 'active', 'cardio'], axis=1).values

y_smoke = train['smoke'].values
y_alco = train['alco'].values
y_active = train['active'].values

In [289]:
# smoke_params = {}
# alco_params = {}
# active_params = {}

In [297]:
smoke_estimator = search_xgb(X_helper, y_smoke, smoke_params)


[29s] -0.2392025 {'base_score': 0.52} | {'base_score': [0.5, 0.51, 0.52]}
[29s] -0.2392025 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.98, 1.0, 1.0]}
[28s] -0.2392025 {'reg_lambda': 0.98} | {'reg_lambda': [0.98, 0.99, 1.0]}
[28s] -0.2392025 {'reg_alpha': 0.009999999999999997} | {'reg_alpha': [0.009999999999999997, 0.019999999999999997, 0.03]}
[28s] -0.2391664 {'gamma': 0.18} | {'gamma': [0.16999999999999998, 0.18, 0.19]}
[35s] -0.2391664 {'max_depth': 4} | {'max_depth': [4, 5, 6]}
[54s] -0.2391581 {'n_estimators': 81} | {'n_estimators': [77, 78, 79, 80, 81, 82, 83]}

[23s] -0.2391581 {'base_score': 0.52} | {'base_score': [0.5, 0.51, 0.52]}
[23s] -0.2391581 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.98, 1.0, 1.0]}
[23s] -0.2391581 {'reg_lambda': 0.98} | {'reg_lambda': [0.98, 0.99, 1.0]}
[23s] -0.2391581 {'reg_alpha': 0.009999999999999997} | {'reg_alpha': [0.009999999999999997, 0.019999999999999997, 0.03]}
[23s] -0.2391581 {'gamma': 0.18} | {'gamma': [0.1699999999999999

In [298]:
alco_estimator = search_xgb(X_helper, y_alco, alco_params)


[21s] -0.1923655 {'base_score': 0.51} | {'base_score': [0.5, 0.51, 0.52]}
[21s] -0.1923655 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.98, 1.0, 1.0]}
[21s] -0.1923655 {'reg_lambda': 1.0} | {'reg_lambda': [0.98, 0.99, 1.0]}
[21s] -0.1924033 {'reg_alpha': 0.009999999999999997} | {'reg_alpha': [0.009999999999999997, 0.019999999999999997, 0.03]}
[21s] -0.1923829 {'gamma': 0.16999999999999998} | {'gamma': [0.16999999999999998, 0.18, 0.19]}
[35s] -0.1925351 {'max_depth': 4} | {'max_depth': [4, 5, 6]}
[55s] -0.1924453 {'n_estimators': 77} | {'n_estimators': [77, 78, 79, 80, 81, 82, 83]}

[23s] -0.1923628 {'base_score': 0.52} | {'base_score': [0.5, 0.51, 0.52]}
[23s] -0.1923628 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.98, 1.0, 1.0]}
[24s] -0.1923628 {'reg_lambda': 1.0} | {'reg_lambda': [0.98, 0.99, 1.0]}
[23s] -0.1923628 {'reg_alpha': 0.009999999999999997} | {'reg_alpha': [0.009999999999999997, 0.019999999999999997, 0.03]}
[23s] -0.1923628 {'gamma': 0.16999999999999998} | {

In [299]:
active_estimator = search_xgb(X_helper, y_active, active_params)


[21s] -0.4928248 {'base_score': 0.5} | {'base_score': [0.5, 0.51, 0.52]}
[21s] -0.4928248 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.98, 1.0, 1.0]}
[20s] -0.4928034 {'reg_lambda': 0.99} | {'reg_lambda': [0.98, 0.99, 1.0]}
[21s] -0.4928468 {'reg_alpha': 0.03} | {'reg_alpha': [0.009999999999999997, 0.019999999999999997, 0.03]}
[21s] -0.4928251 {'gamma': 0.19} | {'gamma': [0.16999999999999998, 0.18, 0.19]}
[33s] -0.4926050 {'max_depth': 4} | {'max_depth': [4, 5, 6]}
[51s] -0.4926262 {'n_estimators': 83} | {'n_estimators': [77, 78, 79, 80, 81, 82, 83]}

[23s] -0.4926055 {'base_score': 0.51} | {'base_score': [0.5, 0.51, 0.52]}
[23s] -0.4925997 {'scale_pos_weight': 0.98} | {'scale_pos_weight': [0.98, 1.0, 1.0]}
[23s] -0.4925997 {'reg_lambda': 0.99} | {'reg_lambda': [0.98, 0.99, 1.0]}
[23s] -0.4925997 {'reg_alpha': 0.03} | {'reg_alpha': [0.009999999999999997, 0.019999999999999997, 0.03]}
[23s] -0.4925953 {'gamma': 0.18} | {'gamma': [0.16999999999999998, 0.18, 0.19]}
[28s] -0.4925953

### Predict `cardio`

In [292]:
'''
xgb_params = {
    'base_score': 0.51,
    'gamma': 0.18,
    'max_depth': 5,
    'n_estimators': 80,
    'reg_alpha': 0.019999999999999997,
    'reg_lambda': 0.99,
    'scale_pos_weight': 1.0,
}
'''

In [300]:
cardio_estimator = search_xgb(X, y, xgb_params)
print_importances(cardio_estimator)


[32s] -0.5384644 {'base_score': 0.51} | {'base_score': [0.5, 0.51, 0.52]}
[32s] -0.5384644 {'scale_pos_weight': 1.0} | {'scale_pos_weight': [0.98, 1.0, 1.0]}
[32s] -0.5384644 {'reg_lambda': 0.99} | {'reg_lambda': [0.98, 0.99, 1.0]}
[32s] -0.5384644 {'reg_alpha': 0.019999999999999997} | {'reg_alpha': [0.009999999999999997, 0.019999999999999997, 0.03]}
[32s] -0.5384644 {'gamma': 0.18} | {'gamma': [0.16999999999999998, 0.18, 0.19]}
[32s] -0.5384644 {'max_depth': 5} | {'max_depth': [4, 5, 6]}
[74s] -0.5384644 {'n_estimators': 80} | {'n_estimators': [77, 78, 79, 80, 81, 82, 83]}
age: 0.2551440
bmi: 0.1627801
ap_hi: 0.1252858
height: 0.1001372
weight: 0.0960220
ap_lo: 0.0759031
cholesterol: 0.0653864
gluc: 0.0480110
active: 0.0301783
smoke: 0.0169182
alco: 0.0132602
gender: 0.0109739


In [314]:
test = read_csv('test.csv')
X_helper_test = test.drop(['smoke', 'alco', 'active'], axis=1).values

test['smoke'].fillna(pandas.Series(smoke_estimator.predict_proba(X_helper_test)[:, 1]), inplace=True)
test['alco'].fillna(pandas.Series(alco_estimator.predict_proba(X_helper_test)[:, 1]), inplace=True)
test['active'].fillna(pandas.Series(active_estimator.predict_proba(X_helper_test)[:, 1]), inplace=True)

numpy.savetxt(f'xgboost.txt', cardio_estimator.predict_proba(test.values)[:, 1], fmt='%f')